In [43]:
# PAPER - dELVING dEEP INTO RECTIFIERS:  SURPASSING HUMAN-LEVEL PERFORMANCE ON IMAGENET CLASSIFICATION BY KAIMING HE

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
# reading all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
words[:8]

['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']

In [3]:
len(words)

29681

In [4]:
# building the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
# building the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80% of the data
Xdev, Ydev = build_dataset(words[n1:n2]) # 10% of the data
Xte, Yte = build_dataset(words[n2:]) # 10% of the data

torch.Size([170379, 3]) torch.Size([170379])
torch.Size([21222, 3]) torch.Size([21222])
torch.Size([21124, 3]) torch.Size([21124])


In [50]:
# MLP revisited
n_embd = 10 # the dimensionality of the embeddings
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((vocab_size, n_embd),          generator=g) # the embedding matrix
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * ((5/3) / (n_embd * block_size)**0.5) # 0.2 # the first layer of the MLP # "Kaiming init"
b1 = torch.randn((n_hidden),                    generator=g) * 0.01 # the bias of the first layer
W2 = torch.randn((n_hidden, vocab_size),        generator=g) * 0.01 # the second layer of the MLP
b2 = torch.randn((vocab_size),                  generator=g) * 0 # the bias of the second layer

parameters = [C, W1, b1, W2, b2]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True


11897


In [46]:
(torch.randn(10000) * 0.2).std() # numbers for gaussian

tensor(0.2026)

In [47]:
n_embd * block_size

30

In [48]:
(5/3) / (30**0.5)

0.3042903097250923

In [51]:
x = torch.randn(1000, 10)
w = torch.randn(10, 200) / 10**0.5
y = x @ w
print(x.mean(), x.std())
print(y.mean(), y.std())


tensor(-0.0090) tensor(1.0008)
tensor(-0.0017) tensor(1.0008)


In [ ]:
# x = torch.randn(1000, 10)
# w = torch.randn(10, 200)
# y = x @ w
# print(x.mean(), x.std())
# print(y.mean(), y.std())
# plt.figure(figsize=(16,4))
# plt.subplot(121)
# plt.hist(x.view(-1.tolist(), 50, density=True));
# plt.subplot(122)
# plt.hist(y.view(-1).tolist(), 50, density=True)

In [52]:
# same optimization as in part 2
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

  # minibatch construction
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  hpreact = embcat @ W1 + b1 # hidden layer pre-activation
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # tracking the stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: loss = {loss.item():.4f}')
  lossi.append(loss.log10().item())

      0/ 200000: loss = 3.3188
  10000/ 200000: loss = 2.0989
  20000/ 200000: loss = 2.3204
  30000/ 200000: loss = 2.1513
  40000/ 200000: loss = 1.9521
  50000/ 200000: loss = 2.2986
  60000/ 200000: loss = 2.6944
  70000/ 200000: loss = 2.2578
  80000/ 200000: loss = 2.2030
  90000/ 200000: loss = 2.7034
 100000/ 200000: loss = 2.0390
 110000/ 200000: loss = 2.0694
 120000/ 200000: loss = 2.1744
 130000/ 200000: loss = 2.4378
 140000/ 200000: loss = 1.8644
 150000/ 200000: loss = 2.2597
 160000/ 200000: loss = 2.0452
 170000/ 200000: loss = 2.0599
 180000/ 200000: loss = 2.1208
 190000/ 200000: loss = 2.3622


In [8]:
# plt.figure(figsize=(20,10))
# plt.imshow(h.abs() > 0.99, cmap='gray', interpolation='nearest')

In [53]:
h

tensor([[-0.9617, -0.9895, -0.8348,  ..., -0.9999,  0.6841,  0.9956],
        [ 0.9962,  0.9574, -0.3894,  ..., -0.9434,  0.8567,  0.9188],
        [-0.9967,  0.9997, -0.9823,  ..., -0.1281, -0.9900, -0.0703],
        ...,
        [ 0.9691, -0.8923, -0.7311,  ..., -0.9639,  0.9984,  0.9995],
        [-0.9963,  0.9669, -0.9437,  ...,  0.9826, -0.9996, -0.7680],
        [-0.7642,  0.6927, -0.9911,  ..., -0.8065,  0.6230,  0.4331]],
       grad_fn=<TanhBackward0>)

In [33]:
h.shape

torch.Size([32, 200])

In [11]:
# plt.hist(h.view(-1).tolist(), 50);

In [12]:
# plt.hist(hpreact.view(-1).tolist(), 50);

In [55]:
logits[0] # logits should be much much closer to zero

tensor([ 4.6037,  2.4816, -2.9437,  0.3146, -0.0422,  2.7464, -0.1512, -0.8929,
        -1.7696,  0.3095, -0.8370,  1.3046,  0.4624, -0.0111,  0.9328, -0.4851,
        -3.7180, -0.5719,  0.4491,  1.8187,  0.9771, -0.1514, -0.5998, -4.3349,
        -0.7540,  2.7375, -0.9183], grad_fn=<SelectBackward0>)

In [35]:
# probability for any character
-torch.tensor(1/27.0).log()

tensor(3.2958)

In [56]:
# 4 dimensional example of the issue
logits = torch.tensor([1.0, 1.0, 1.0, 1.0])
# logits = torch.randn(4) * 10
probs = torch.softmax(logits, dim=0)
loss = -probs[2].log()
logits, probs, loss

(tensor([1., 1., 1., 1.]),
 tensor([0.2500, 0.2500, 0.2500, 0.2500]),
 tensor(1.3863))

In [16]:
# plt.plot(lossi)

In [57]:
@torch.no_grad() # this ensures that we don't save gradients
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte)
  }[split]
  emb = C[x] # (N, block_size, n_embd) embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors (N, block_size * n_embd)
  h = torch.tanh(embcat @ W1 + b1) # hidden layer (N, n_hidden)
  logits = h @ W2 + b2 # output layer (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())
  
split_loss('train')
split_loss('val')
split_loss('test')

train 2.0455400943756104
val 2.1180968284606934
test 2.128801107406616


In [60]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10) # for reproducibility

for _ in range(20):

  out = []
  context = [0] * block_size # initialize with all dots
  while True:
    # forward pass
    emb = C[torch.tensor([context])] # embedding
    h = torch.tanh(emb.view(1, -1) @ W1 + b1) # hidden layer
    logits = h @ W2 + b2 # output layer
    probs = F.softmax(logits, dim=1) # probabilities
    
    # sample from the distribution
    ix = torch.multinomial(probs, num_samples=1, generator=g).item()
    
    # shift the context window and track the sample
    context = context[1:] + [ix]
    out.append(ix)
    
    # stop if we reach the end
    if ix == 0:
      break

  print(''.join(itos[i] for i in out)) # decode and print the sample

montaymyah.
seel.
ndhayla.
reivan.
endra.
gradee.
daelin.
shi.
jen.
eden.
van.
aarielle.
malaia.
noshubergihiriel.
kindreelynn.
novana.
ubamard.
ryyah.
fael.
yuma.
